In [75]:
!pip install webdriver-manager
!pip install -U sentence-transformers


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [490]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

import time
import numpy as np
import pandas as pd
import multiprocessing
import re
from rapidfuzz import process
import unicodedata

from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
from sklearn.metrics import jaccard_score
from rapidfuzz.distance.Levenshtein import normalized_similarity

# Scraping Linkedin

In [77]:
job_query = "python-developer"
len_jobs = 30

In [78]:
start = time.time()
# driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver = webdriver.Chrome()
url = f"https://ph.linkedin.com/jobs/{job_query}-jobs"

try:
    driver.get(url)
    ActionChains(driver).send_keys(Keys.ESCAPE).perform()
    job_list = driver.find_elements(By.XPATH, "//ul[@class='jobs-search__results-list']/li")
    
    # job catalog scraping
    jobs_scraped = np.array([])
    for job in job_list:
        descs = []
        try:
            job_title = job.find_element(By.XPATH, ".//h3[@class='base-search-card__title']").get_attribute('innerHTML').strip()
            card = job.find_element(By.XPATH, ".//a[@data-tracking-control-name='public_jobs_jserp-result_search-card']")
            link = card.get_attribute('href')
            descs = [job_title, link]
        except:
            continue
    
        try:
            location = job.find_element(By.XPATH, ".//span[@class='job-search-card__location']").get_attribute('innerHTML')
            location = location.replace('\n', '').strip()
            descs.append(location)
        except:
            descs.append('')
        
        try: 
            company = job.find_element(By.XPATH, ".//h4[@class='base-search-card__subtitle']/a").get_attribute('innerHTML').strip()
            descs.append(company)
        except:
            try:
                company = job.find_element(By.XPATH, ".//h4[@class='base-search-card__subtitle']").get_attribute('innerHTML').strip()
                descs.append(company)
            except:
                descs.append('')
                
        if len(jobs_scraped) == 0:
            jobs_scraped = np.append(jobs_scraped, descs)
        else:
            jobs_scraped = np.vstack([jobs_scraped, descs])

        # limit to max len_jobs only
        if len(jobs_scraped) >= len_jobs:
            break
    
    # if there is only 1 scraped job
    if jobs_scraped.shape == (4,):
        jobs_scraped = np.array([list(jobs_scraped)])
        
    # individual job scraping
    job_descs = np.array([])
    for job in jobs_scraped:
        descs = []
        try:
            # scraping emp_type, job_func, job_desc, posted ago
            driver.get(job[1])
            wait = WebDriverWait(driver, timeout=2)
            desc_job = wait.until(EC.presence_of_all_elements_located((By.XPATH, ".//ul[@class='description__job-criteria-list']/li")))
            descs = [job[1]]
            for i in [1,2]:
                try:
                    detail = desc_job[i].find_element(By.XPATH, ".//span")
                    detail = detail.get_attribute('innerHTML').replace('\n', '').strip()
                    descs.append(detail)
                except:
                    descs.append('')
            try:
                desc_gen = driver.find_element(By.XPATH, "//div[@class='description__text description__text--rich']/section/div")
                desc_gen = desc_gen.get_attribute('innerHTML')
                descs.append(desc_gen)
            except:
                descs.append('')
            try:
                posted_ago = driver.find_element(By.XPATH, "//span[@class='posted-time-ago__text topcard__flavor--metadata']")
                posted_ago = posted_ago.get_attribute('innerHTML').replace('\n','').strip()
                descs.append(posted_ago)
            except:
                descs.append('')
        except: 
            descs = [job[3],'','','','']
        if len(job_descs) == 0:
            job_descs = np.append(job_descs, descs)
        else:
            job_descs = np.vstack([job_descs, descs])
        time.sleep(2)
    
    # merging
    jobs_df = pd.DataFrame(jobs_scraped)
    jobs_df.columns = ['title','link', 'location', 'company']
    job_descs_df = pd.DataFrame(job_descs)
    
    # if there is only 1 scraped job
    if job_descs_df.shape == (5,1):
        job_descs_df = job_descs_df.T
    
    job_descs_df.columns = ['link','emp_type', 'job_func', 'job_desc', 'posted']
    linkedin_df = jobs_df.merge(job_descs_df, on='link', how='left')
except:
    print("Unable to Scrape Linkedin")
finally:
    # close driver
    try:
        driver.close()
    except:
        print("Unable to Scrape Linkedin")
    
end = time.time()
print("TOTAL TIME", end-start)

TOTAL TIME 120.08346915245056


# Scraping FoundIt

In [79]:
job_query = "python-developer"
len_jobs = 30

In [80]:
start = time.time()
# driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver = webdriver.Chrome()
url = f"https://www.foundit.com.ph/search/{job_query}-jobs"

try:
    jobs_scraped = np.array([])
    page_num = 1
    while True:
        driver.get(f"{url}{f"-{page_num}" if page_num > 1 else ''}")
        job_list = driver.find_elements(By.XPATH, "//div[@class='srpResultCard']/div")
        if len(job_list) == 0:
            break
        job_list = job_list[1:] # remove the header
        for job in job_list:
            # scraping title, company, url
            try:
                job_title = job.find_element(By.XPATH, ".//a[@title]")
                link = job_title.get_attribute('href')
                job_title = job_title.get_attribute('innerHTML').replace('\n', '').strip()
            except:
                continue
            try:
                company = job.find_element(By.XPATH, ".//div[@class='companyName']/span")
                company = company.get_attribute('innerHTML').replace('\n', '').strip()
            except: 
                company = ''
            descs = [job_title, company, link]
        
            # scraping location, posted ago
            job.find_element(By.XPATH, './/div[@onclick]/div').click()
            wait = WebDriverWait(driver, 5)
            try:
                desc_job = wait.until(EC.presence_of_all_elements_located((By.XPATH, ".//div[@id='jobHighlight']/div/div/div")))
                for i in [0,2]:
                    if i != 2:
                        detail = desc_job[i].find_element(By.XPATH, ".//div[@class='details']")
                        detail = detail.get_attribute('innerHTML').replace('\n', '').strip()
                    else: 
                        detail = desc_job[i].find_element(By.XPATH, ".//span[@class='btnHighighlights']")
                        detail = detail.get_attribute('innerHTML').replace('\n', '').split('</i>')
                        detail = detail[1].strip()
                    descs.append(detail)
            except:
                descs.append('','','')
        
            # scraping emp_type, job function, general job_desc
            try:
                desc_job_2 = wait.until(EC.visibility_of_all_elements_located((By.XPATH, ".//div[@id='jobDetail']/div/div")))
                for i in [0,2]:
                    try:
                        detail = desc_job_2[i].find_element(By.XPATH, ".//div[@class='jobDesc']")
                        detail = detail.get_attribute('innerHTML').replace('\n', '').strip()
                        descs.append(detail)
                    except:
                        descs.append('')
            except:
                descs.extend(['',''])
            try:
                desc_gen = wait.until(EC.visibility_of_element_located((By.XPATH, ".//p[@class='jobDescInfo']")))
                desc_gen = desc_gen.get_attribute('innerHTML')
                descs.append(desc_gen)
            except:
                descs.append('')
                
            if len(jobs_scraped) == 0:
                jobs_scraped = np.append(jobs_scraped, descs)
            else:
                jobs_scraped = np.vstack([jobs_scraped, descs])

        # limit to max len_jobs only
        if len(jobs_scraped) >= len_jobs:
            break
        page_num += 1

    # converting to DataFrame
    if jobs_scraped.shape == (8,):
        jobs_scraped = np.array([list(jobs_scraped)])
    
    foundit_df = pd.DataFrame(jobs_scraped)
    foundit_df.columns = ['title','company','link','location','posted','emp_type','job_func','job_desc']
except:
    print("Unable to Scrape Foundit")
finally:
    # close driver
    try:
        driver.close()
    except:
        print("Unable to Scrape Foundit")
    
end = time.time()
print("TOTAL TIME", end-start)

TOTAL TIME 37.00051712989807


# Scraping Jobstreet

In [81]:
job_query = "python-developer"
len_jobs = 30

In [82]:
start = time.time()
# driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver = webdriver.Chrome()
url = f"https://ph.jobstreet.com/{job_query}-jobs"

try:
    driver.get(url)
    
    # job catalog scraping
    jobs_scraped = np.array([])
    wait = WebDriverWait(driver, 5)
    while True:
        try:
            job_list = wait.until(EC.presence_of_all_elements_located((By.XPATH, "//article[@data-automation='normalJob']")))
            for job in job_list:
                descs = []
                # scraping title, company, link
                try:
                    job_title = job.find_element(By.XPATH, ".//a[@data-automation='jobTitle']")
                    link = job_title.get_attribute('href')
                    job_title = job_title.get_attribute('innerHTML').replace('\n','').strip()
                    company = job.find_element(By.XPATH, ".//a[@data-type='company']")
                    company = company.get_attribute('innerHTML').replace('\n', '').strip()
                    descs.append(job_title)
                    descs.append(link)
                    descs.append(company)
                except:
                    continue

                try:
                    posted_ago = job.find_element(By.XPATH, ".//span[@data-automation='jobListingDate']")
                    posted_ago = posted_ago.get_attribute('innerHTML').replace('\n','').strip()
                    descs.append(posted_ago)
                except:
                    descs.append('')
                
                if len(jobs_scraped) == 0:
                    jobs_scraped = np.append(jobs_scraped, descs)
                else:
                    jobs_scraped = np.vstack([jobs_scraped, descs])
                    
            # limit to max len_jobs only
            if len(jobs_scraped) >= len_jobs:
                jobs_scraped = jobs_scraped[:len_jobs]
                break
                
            next_button = driver.find_element(By.XPATH, ".//a[@aria-label='Next']")
            next_button.click()
        except:
            break
    
    if jobs_scraped.shape == (4,):
        jobs_scraped = np.array([list(jobs_scraped)])
    
    # individual job scraping
    job_descs = np.array([])
    for job in jobs_scraped:
        try:
            driver.get(job[1])
            descs = [job[1]]
            try:
                location = driver.find_element(By.XPATH, "//span[@data-automation='job-detail-location']/a")
                location = location.get_attribute('innerHTML').replace('\n','').strip()
                descs.append(location)
            except:
                descs.append('')
            try:
                job_func = driver.find_element(By.XPATH, "//span[@data-automation='job-detail-classifications']/a")
                job_func = job_func.get_attribute('innerHTML').replace('\n','').strip()
                descs.append(job_func)
            except:
                descs.append('')
            try:
                emp_type = driver.find_element(By.XPATH, "//span[@data-automation='job-detail-work-type']/a")
                emp_type = emp_type.get_attribute('innerHTML').replace('\n','').strip()
                descs.append(emp_type)
            except:
                descs.append('')
            try:
                job_desc = driver.find_element(By.XPATH, "//div[@data-automation='jobAdDetails']/div")
                job_desc = job_desc.get_attribute('innerHTML').replace('\n','').strip()
                descs.append(job_desc)
            except:
                descs.append('')
        except:
            descs = [job[2], '', '', '', '']
        if len(job_descs) == 0:
            job_descs = np.append(job_descs, descs)
        else:
            job_descs = np.vstack([job_descs, descs])
    
    # merging
    jobs_df = pd.DataFrame(jobs_scraped)
    jobs_df.columns = ['title', 'link', 'company', 'posted']
    job_descs_df = pd.DataFrame(job_descs)
    
    # if there is only 1 scraped job
    if job_descs_df.shape == (5,1):
        job_descs_df = job_descs_df.T
    
    job_descs_df.columns = ['link', 'location', 'job_func', 'emp_type', 'job_desc']
    jobstreet_df = jobs_df.merge(job_descs_df, on='link', how='left')
except:
    print("Unable to Scrape Jobstreet")
finally:
    # close the driver
    try:
        driver.close()
    except:
        print("Unable to Scrape Jobstreet")
end = time.time()
print("TOTAL TIME", end-start)

TOTAL TIME 55.34612822532654


# Scraping Kalibrr

In [83]:
job_query = "python-developer"
len_jobs = 30

In [84]:
start = time.time()
# driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver = webdriver.Chrome()
url = f"https://www.kalibrr.com/home/te/{job_query}"

try:
    driver.get(url)
    job_list = []
    # load job list 
    while len(job_list) < len_jobs:
        # click button to load more jobs until job_list > len_jobs
        try:
            # Re-evaluate job_list after each load
            wait = WebDriverWait(driver, 5)
            job_list = wait.until(EC.presence_of_all_elements_located((By.XPATH, "//div[@class='k-font-dm-sans k-rounded-lg k-bg-white k-border-solid k-border hover:k-border-2 hover:k-border-primary-color k-border k-group k-flex k-flex-col k-justify-between css-1otdiuc']")))
            
            load_more = wait.until(EC.presence_of_element_located((By.XPATH, "//button[@class='k-btn-primary']")))
            load_more.click()
        except:
            break
    job_list[:len_jobs]
    
    # job catalog scraping
    jobs_scraped = np.array([])
    i = 0
    for job in job_list:
        i += 1
        descs = []
        # scraping job title, company, emp_type, location
        try:
            job_title = job.find_element(By.XPATH, ".//h2[@data-tooltip-id='job-title-tooltip-[object Object]']/a")
            url = job_title.get_attribute('href')
            job_title = job_title.get_attribute('innerHTML').replace('\n', '').strip()
            descs.append(job_title)
            descs.append(url)
        except:
            continue
    
        try:
            company = job.find_element(By.XPATH, ".//span[@class='k-inline-flex k-items-center k-mb-1']/a")
            company = company.text.replace('\n', '').strip()
            descs.append(company)
        except:
            descs.append('')
            
        try:
            emp_type = job.find_element(By.XPATH, "./div[@class='k-relative']/div/span/span[@class='k-text-gray-500']")
            emp_type = emp_type.get_attribute('innerHTML').replace('\n', '').strip()
            descs.append(emp_type)
        except:
            descs.append('')
    
        try:
            location = job.find_element(By.XPATH, "./div[@class='k-relative']/div/span/span[@class='k-text-gray-500 k-block k-pointer-events-none']")
            location = location.get_attribute('innerHTML').replace('\n', '').strip()
            descs.append(location)
        except:
            descs.append('')
            
        if len(jobs_scraped) == 0:
            jobs_scraped = np.append(jobs_scraped, descs)
        else:
            jobs_scraped = np.vstack([jobs_scraped, descs])

        # limit to len_jobs only
        if len(jobs_scraped) >= len_jobs:
            break
    
    if jobs_scraped.shape == (5,):
        jobs_scraped = np.array([list(jobs_scraped)])
    
    # individual job scraping
    job_descs = np.array([])
    for job in jobs_scraped:
        descs = [job[1]]
        try:
            driver.get(job[1])
            # scrape job_func, posted
            try:
                wait = WebDriverWait(driver, timeout=2)
                job_func = wait.until(EC.presence_of_element_located((By.XPATH, ".//div[@class='md:k-flex']//dt[contains(text(),'Job Category')]/following-sibling::dd/a")))
                job_func = job_func.get_attribute('innerHTML').replace('\n', '').strip()
                descs.append(job_func)
            except:
                descs.append('')
            try:
                posted = driver.find_element(By.XPATH, ".//div[@class='k-text-subdued k-text-caption md:k-text-right md:k-absolute md:k-right-0 md:k-top-0 md:k-p-4']/p")
                posted = posted.get_attribute('innerHTML').replace('\n', '').strip()
                descs.append(posted)
            except:
                descs.append('')
            # scrape desc
            job_desc = ''
            try:
                job_desc = driver.find_element(By.XPATH, ".//div[@itemprop='description']")
                job_desc = job_desc.get_attribute('innerHTML')
                job_desc += job_desc
            except:
                job_desc = job_desc
            try:
                job_qual = driver.find_element(By.XPATH, ".//div[@itemprop='qualifications']")
                job_qual = job_qual.get_attribute('innerHTML')
                job_desc += job_qual
            except:
                job_desc = job_desc
            try:
                job_benef = driver.find_element(By.XPATH, ".//div[@itemprop='jobBenefits']")
                job_benef = job_benef.get_attribute('innerHTML').replace('\n', '').strip()
                job_desc += job_qual
            except:
                job_desc = job_desc
            try:
                job_skills = driver.find_element(By.XPATH, ".//ul")
                job_skills = job_skills.get_attribute('innerHTML')
                job_desc += job_skills
            except:
                job_desc = job_desc
        
            descs.append(job_desc)
        
            if len(job_descs) == 0:
                job_descs = np.append(job_descs, descs)
            else:
                job_descs = np.vstack([job_descs, descs])
        except:
            continue
    
    jobs_df = pd.DataFrame(jobs_scraped)
    jobs_df.columns = ['title', 'link' ,'company', 'emp_type', 'location']
    
    job_descs_df = pd.DataFrame(job_descs)
    
    if job_descs_df.shape == (3, 1):
        job_descs_df = job_descs_df.T
    job_descs_df.columns = ['link', 'job_func', 'posted', 'job_desc']
    kalibrr_df = jobs_df.merge(job_descs_df, on='link', how='left')

except:
    print("Unable to Scrape Kalibrr")
finally:
    # close driver
    try:
        driver.close()
    except:
        print("Unable to Scrape Kalibrr")

end = time.time()
print("TOTAL TIME", end-start)    

TOTAL TIME 34.61245584487915


# Checkpoint

In [85]:
# saving
linkedin_df.to_csv('linkedin.csv')
foundit_df.to_csv('foundit.csv')
jobstreet_df.to_csv('jobstreet.csv')
kalibrr_df.to_csv('kalibrr.csv')

In [70]:
# loading
linkedin_df = pd.read_csv('linkedin.csv')
foundit_df = pd.read_csv('foundit.csv')
jobstreet_df = pd.read_csv('jobstreet.csv')
kalibrr_df = pd.read_csv('kalibrr.csv')

# Preprocessing

In [71]:
linkedin_df.columns

Index(['Unnamed: 0', 'title', 'link', 'location', 'company', 'emp_type',
       'job_func', 'job_desc', 'posted'],
      dtype='object')

In [72]:
foundit_df.columns

Index(['Unnamed: 0', 'title', 'company', 'link', 'location', 'posted',
       'emp_type', 'job_func', 'job_desc'],
      dtype='object')

In [73]:
jobstreet_df.columns

Index(['Unnamed: 0', 'title', 'link', 'company', 'posted', 'location',
       'job_func', 'emp_type', 'job_desc'],
      dtype='object')

In [74]:
kalibrr_df.columns

Index(['Unnamed: 0', 'title', 'link', 'company', 'emp_type', 'location',
       'job_func', 'posted', 'job_desc'],
      dtype='object')

In [75]:
# Drop 'Unnamed: 0' column
linkedin_df = linkedin_df.drop(columns=['Unnamed: 0'], errors='ignore')
foundit_df = foundit_df.drop(columns=['Unnamed: 0'], errors='ignore')
jobstreet_df = jobstreet_df.drop(columns=['Unnamed: 0'], errors='ignore')
kalibrr_df = kalibrr_df.drop(columns=['Unnamed: 0'], errors='ignore')

## Standardizing Job Functions

In [76]:
sheet_id = "1FJgg2JWrKfzyWbi-76vKcR1dm-q6RwCAipobzotr_Eg"
sheet_name = "Job_Functions"

url= f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"

df_jobs = pd.read_csv(url)

df_jobs

,Standardized,Linkedin,Foundit,Kalibrr,Jobstreet
0,Accounting & Auditing,Accounting/Auditing,NaN,Accounting and Finance,Accounting
1,Administration & Office Support,Administrative,Admin/secretarial/front office,Adminstration and Coordination,Administration & Office Support
2,Advertising,NaN,Advertising/entertainment/media,Media and Creatives,"Advertising, Arts & Media"
3,Analyst,Analyst,Analytics/business intelligence,NaN,NaN
4,Architecture,NaN,Architecture/interior design,Architecture and Engineering,Design & Architecture
...,...,...,...,...,...
64,Sales,NaN,Sales/business development,NaN,NaN
65,Sales,Entrepreneurship,Retail chains,NaN,NaN
66,Sales,NaN,Fashion/apparels,NaN,NaN
67,Sciences,Research,NaN,Sciences,Science & Technology


In [77]:
# mapping and standardizing
def standardization_map_exact(df, source_col, standard_col='Standardized'):
    source = df.dropna(subset=[source_col, standard_col])[source_col]
    target = df.dropna(subset=[source_col, standard_col])[standard_col]
    mapping = dict(zip(source, target))
    return mapping


def standardize_job_function(df_jobs_ref, df_target, site_column, source_col, target_col='job_func_stand'):
    # Create the mapping dictionary from the job mapping table
    mapping_dict = standardization_map_exact(df_jobs_ref,site_column)
    
    # Apply the mapping to the job function column
    df_target[target_col] = df_target[source_col].apply(
        lambda x: ", ".join(
            mapping_dict.get(f.strip(), f.strip()) 
            for f in str(x).split(",") if f.strip()
        )
    )

### Linkedin, Kalibrr, and Jobstreet

In [78]:
standardize_job_function(df_jobs, linkedin_df, 'Linkedin', 'job_func')
standardize_job_function(df_jobs, kalibrr_df, 'Kalibrr', 'job_func')

jobstreet_df['job_func_clean'] = jobstreet_df['job_func'].str.extract(r'\(([^)]+)\)')
jobstreet_df['job_func_clean'] = jobstreet_df['job_func_clean'].str.replace('&amp;', '&', regex=False)

standardize_job_function(df_jobs, jobstreet_df, 'Jobstreet', 'job_func_clean')

linkedin_df[['job_func', 'job_func_stand']].head()

,job_func,job_func_stand
0,"Engineering, Information Technology","Engineering, Information Technology"
1,"Engineering, Information Technology","Engineering, Information Technology"
2,"Engineering, Information Technology","Engineering, Information Technology"
3,"Engineering, Information Technology","Engineering, Information Technology"
4,"Engineering, Information Technology","Engineering, Information Technology"


In [79]:
kalibrr_df[['job_func', 'job_func_stand']].head()

,job_func,job_func_stand
0,IT and Software,Information Technology
1,IT and Software,Information Technology
2,IT and Software,Information Technology
3,IT and Software,Information Technology
4,IT and Software,Information Technology


In [80]:
jobstreet_df[['job_func_clean', 'job_func_stand']].head()

,job_func_clean,job_func_stand
0,Information & Communication Technology,Information Technology
1,Information & Communication Technology,Information Technology
2,Information & Communication Technology,Information Technology
3,Information & Communication Technology,Information Technology
4,Information & Communication Technology,Information Technology


## Standardizing Location

In [81]:
sheet_id = "1FJgg2JWrKfzyWbi-76vKcR1dm-q6RwCAipobzotr_Eg"
sheet_name = "LocationwCoordinates"

url= f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"

df_locations = pd.read_csv(url)


df_locations['city_clean'] = df_locations['City/Province'].str.replace(r'\s+City$', '', regex=True)


df_locations

,City/Province,Region,Type,Latitude,Longitude,city_clean
0,Caloocan City,National Capital Region (NCR),City,14.6500,120.9667,Caloocan
1,Las Pinas City,National Capital Region (NCR),City,14.6333,121.0333,Las Pinas
2,Makati City,National Capital Region (NCR),City,14.5503,121.0327,Makati
3,Malabon City,National Capital Region (NCR),City,14.6600,120.9600,Malabon
4,Mandaluyong City,National Capital Region (NCR),City,14.6167,121.0333,Mandaluyong
...,...,...,...,...,...,...
211,Lanao del Sur,Bangsamoro Autonomous Region in Muslim Mindana...,Province,7.9167,124.3000,Lanao del Sur
212,Sulu,Bangsamoro Autonomous Region in Muslim Mindana...,Province,6.0000,121.0000,Sulu
213,Tawi-Tawi,Bangsamoro Autonomous Region in Muslim Mindana...,Province,5.1333,120.1000,Tawi-Tawi
214,Maguindanao del Norte,Bangsamoro Autonomous Region in Muslim Mindana...,Province,7.1833,124.4333,Maguindanao del Norte


In [82]:
def standardize_and_add_coordinates(df_target, location_col, df_locations, new_col='location_standardized'):
    def normalize(text):
        text = str(text).lower().replace(',', '').replace('-', '').replace('philippines', '').strip()
        text = unicodedata.normalize('NFKD', text).encode('ASCII', 'ignore').decode() 
        return text

    def smart_location_match(raw_location, keywords, df_lookup):
        raw = normalize(raw_location)
        if raw_location.lower().strip() == "philippines" or raw == "other":
            return "Philippines"

        for keyword in keywords:
            if normalize(keyword) in raw:
                match = keyword
                break
        else:
            match, score, _ = process.extractOne(raw, keywords, processor=normalize)
            if score <= 80:
                return "Other"

        if match in df_lookup['city_clean'].values:
            matches = df_lookup[df_lookup['city_clean'] == match]
            region = matches['Region'].values[0]
            type_ = matches['Type'].values[0]
            if type_ == 'City':
                return f"{match.title()} City, {region}"
            else:
                return f"{match.title()}, {region}"
        elif match in df_lookup['Region'].values:
            return match
        else:
            return "Other"

    # Build keywords from cities + regions
    keywords = pd.concat([df_locations['city_clean'], df_locations['Region']]).dropna().unique().tolist()

    # Standardize location
    df_target[new_col] = df_target[location_col].apply(lambda x: smart_location_match(x, keywords, df_locations))

    # Extract city_clean for merge
    df_target['city_clean_match'] = df_target[new_col].apply(
        lambda x: x.split(",")[0].replace(" City", "").strip().lower()
    )

    # Prepare reference for matching
    df_locations_temp = df_locations.copy()
    df_locations_temp['city_clean'] = df_locations_temp['city_clean'].str.lower()

    # Remove old lat/lon if any
    df_target = df_target.drop(columns=['Latitude', 'Longitude'], errors='ignore')

    # Merge lat/lon
    df_target = df_target.merge(
        df_locations_temp[['city_clean', 'Latitude', 'Longitude']],
        how='left',
        left_on='city_clean_match',
        right_on='city_clean'
    )

    # Add coordinates for "Philippines"
    philippines_mask = df_target[new_col] == "Philippines"
    df_target.loc[philippines_mask, "Latitude"] = 12.8797
    df_target.loc[philippines_mask, "Longitude"] = 121.7740

    # Clean up
    df_target.drop(columns=['city_clean_match', 'city_clean'], inplace=True, errors='ignore')

    return df_target

### Linkedin, Foundit, Kalibrr, and Jobstreet

In [83]:
linkedin_df = standardize_and_add_coordinates(linkedin_df, location_col='location', df_locations=df_locations)
foundit_df = standardize_and_add_coordinates(foundit_df, location_col='location', df_locations=df_locations)
kalibrr_df = standardize_and_add_coordinates(kalibrr_df, location_col='location', df_locations=df_locations)
jobstreet_df = standardize_and_add_coordinates(jobstreet_df, location_col='location', df_locations=df_locations)

linkedin_df['location_standardized'].value_counts()

location_standardized
Manila City, National Capital Region (NCR)    16
Philippines                                    6
Pasig City, National Capital Region (NCR)      2
Quezon City, National Capital Region (NCR)     1
National Capital Region (NCR)                  1
Makati City, National Capital Region (NCR)     1
Taguig City, National Capital Region (NCR)     1
Cebu City, Central Visayas (Region VII)        1
Pasay City, National Capital Region (NCR)      1
Name: count, dtype: int64

In [84]:
foundit_df['location_standardized'].value_counts()

location_standardized
Philippines                                   13
Taguig City, National Capital Region (NCR)     6
Manila City, National Capital Region (NCR)     5
Quezon City, National Capital Region (NCR)     4
Makati City, National Capital Region (NCR)     2
Central Visayas (Region VII)                   1
Name: count, dtype: int64

In [85]:
kalibrr_df['location_standardized'].value_counts()

location_standardized
Manila City, National Capital Region (NCR)         12
Quezon City, National Capital Region (NCR)          5
Makati City, National Capital Region (NCR)          4
Paranaque City, National Capital Region (NCR)       3
Mandaluyong City, National Capital Region (NCR)     3
Pasig City, National Capital Region (NCR)           3
Taguig City, National Capital Region (NCR)          1
Name: count, dtype: int64

In [86]:
jobstreet_df['location_standardized'].value_counts()

location_standardized
Manila City, National Capital Region (NCR)         21
Makati City, National Capital Region (NCR)          3
Cebu City, Central Visayas (Region VII)             2
Rizal, CALABARZON (Region IV-A)                     1
Laguna, CALABARZON (Region IV-A)                    1
Calamba City, CALABARZON (Region IV-A)              1
Mandaluyong City, National Capital Region (NCR)     1
Name: count, dtype: int64

In [87]:
# after preprocessing
loc_count = pd.concat([linkedin_df['location_standardized'].value_counts(),foundit_df['location_standardized'].value_counts(),kalibrr_df['location_standardized'].value_counts(), jobstreet_df['location_standardized'].value_counts()],axis=1)
loc_count.columns = ['Linkedin', 'Foundit', 'Kalibrr', 'Jobstreet']
loc_count

,Linkedin,Foundit,Kalibrr,Jobstreet
location_standardized,,,,
"Manila City, National Capital Region (NCR)",16.0,5.0,12.0,21.0
Philippines,6.0,13.0,NaN,NaN
"Pasig City, National Capital Region (NCR)",2.0,NaN,3.0,NaN
"Quezon City, National Capital Region (NCR)",1.0,4.0,5.0,NaN
National Capital Region (NCR),1.0,NaN,NaN,NaN
"Makati City, National Capital Region (NCR)",1.0,2.0,4.0,3.0
"Taguig City, National Capital Region (NCR)",1.0,6.0,1.0,NaN
"Cebu City, Central Visayas (Region VII)",1.0,NaN,NaN,2.0
"Pasay City, National Capital Region (NCR)",1.0,NaN,NaN,NaN


## Standardizing Job Type

In [88]:
sheet_id = "1FJgg2JWrKfzyWbi-76vKcR1dm-q6RwCAipobzotr_Eg"
sheet_name = "Job_Type"

url= f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"

df_job_type = pd.read_csv(url)

df_job_type

,Standardized,Linkedin,Foundit,Kalibrr,Jobstreet
0,Full-time,Full-time,Permanent Job,Full time,Full time
1,Contract/Temporary,Contract,Contract Job,Contractual,Contract/Temp
2,Internship,Internship,NaN,NaN,NaN
3,Part-time,Part-time,NaN,Part time,Part time
4,Contract/Temporary,Temporary,NaN,Freelance,Casual/Vacation


In [89]:
# clean comma-separated job types
def clean_job_type(val):
    job_types = [t.strip() for t in str(val).split(',')]
    
    remove = {'Other','Work From Home', 'Jobs for Women'}
    
    cleaned = [x for x in job_types if x not in remove]
    
    return ", ".join(cleaned) if cleaned else None

# infer job type from job title
def job_type_from_title(df, target_col='job_type_stand', text_col='title'):

    # intern & internship == Internship; part time & part-time == Part-time
    keywords = {r'\bintern\b|\binternship\b': 'Internship', r'\bpart-time\b|\bpart time\b': 'Part-time'}

    for pattern in keywords:
        job_type = keywords[pattern]

        is_match = df[text_col].str.contains(pattern, case=False, na=False)

        df.loc[is_match, target_col] = job_type
    
    return df


def standardize_job_type(df, mapping_dict, source_col):
    df['job_type_stand'] = df[source_col].apply(
        lambda x: ", ".join(
            mapping_dict.get(f.strip(), f.strip()) 
            for f in str(x).split(",") if f.strip()
        )
    )

# entire preprocessing function
def preprocess_job_type(df, source_name, source_col='emp_type', standard_col='Standardized'):
   
    df[source_col] = df[source_col].apply(clean_job_type)
    
    df = job_type_from_title(df, target_col='job_type_stand', text_col='title')
    
    mapping = standardization_map_exact(df_job_type, source_name, standard_col)
    
    standardize_job_type(df, mapping, source_col)
    
    return df



In [90]:
# before preprocessing
job_type_count = pd.concat([linkedin_df['emp_type'].value_counts(),foundit_df['emp_type'].value_counts(),kalibrr_df['emp_type'].value_counts(), jobstreet_df['emp_type'].value_counts()],axis=1)
job_type_count.columns = ['Linkedin', 'Foundit', 'Kalibrr', 'Jobstreet']
job_type_count

,Linkedin,Foundit,Kalibrr,Jobstreet
emp_type,,,,
Full-time,28.0,NaN,NaN,NaN
Contract,1.0,NaN,NaN,NaN
Permanent Job,NaN,31.0,NaN,NaN
Full time,NaN,NaN,30.0,30.0
Freelance,NaN,NaN,1.0,NaN


In [91]:
# preprocessing
linkedin_df = preprocess_job_type(linkedin_df, 'Linkedin')
foundit_df = preprocess_job_type(foundit_df, 'Foundit')
kalibrr_df = preprocess_job_type(kalibrr_df, 'Kalibrr')
jobstreet_df = preprocess_job_type(jobstreet_df, 'Jobstreet')

In [92]:
# after preprocessing
job_type_count = pd.concat([linkedin_df['job_type_stand'].value_counts(),foundit_df['job_type_stand'].value_counts(),kalibrr_df['job_type_stand'].value_counts(), jobstreet_df['job_type_stand'].value_counts()],axis=1)
job_type_count.columns = ['Linkedin', 'Foundit', 'Kalibrr', 'Jobstreet']
job_type_count

,Linkedin,Foundit,Kalibrr,Jobstreet
job_type_stand,,,,
Full-time,28,31.0,30.0,30.0
nan,1,NaN,NaN,NaN
Contract/Temporary,1,NaN,1.0,NaN


In [93]:
linkedin_df_stand = linkedin_df.drop(['job_func','location','emp_type'], axis=1)
kalibrr_df_stand = kalibrr_df.drop(['job_func','location','emp_type'], axis=1)
# foundit_df_stand = foundit_df.drop(['job_func','location'], axis=1)
foundit_df_stand = foundit_df.drop(['location','emp_type'], axis=1)
jobstreet_df_stand = jobstreet_df.drop(['job_func','job_func_clean','location','emp_type'], axis=1)

## Merging the Job Listings

In [94]:
linkedin_df_stand.columns

Index(['title', 'link', 'company', 'job_desc', 'posted', 'job_func_stand',
       'location_standardized', 'Latitude', 'Longitude', 'job_type_stand'],
      dtype='object')

In [95]:
kalibrr_df_stand.columns

Index(['title', 'link', 'company', 'posted', 'job_desc', 'job_func_stand',
       'location_standardized', 'Latitude', 'Longitude', 'job_type_stand'],
      dtype='object')

In [96]:
foundit_df_stand.columns

Index(['title', 'company', 'link', 'posted', 'job_func', 'job_desc',
       'location_standardized', 'Latitude', 'Longitude', 'job_type_stand'],
      dtype='object')

In [97]:
jobstreet_df_stand.columns

Index(['title', 'link', 'company', 'posted', 'job_desc', 'job_func_stand',
       'location_standardized', 'Latitude', 'Longitude', 'job_type_stand'],
      dtype='object')

In [98]:
# merge the four dataframes
columns = ['title', 'company', 'link', 'job_desc','job_func_stand', 'location_standardized', 'job_type_stand']

linkedin_df_stand['source'] = 'Linkedin'
kalibrr_df_stand['source'] = 'Kalibrr'
foundit_df_stand['source'] = 'Foundit'
jobstreet_df_stand['source'] = 'Jobstreet'

for df in [linkedin_df_stand, kalibrr_df_stand, foundit_df_stand, jobstreet_df_stand]:
    for col in columns:
        if col not in df.columns:
            df[col] = np.nan


job_listings = pd.concat([
    linkedin_df_stand[columns + ['source']],
    kalibrr_df_stand[columns + ['source']],
    foundit_df_stand[columns + ['source']],
    jobstreet_df_stand[columns + ['source']]
], ignore_index=True)

job_listings

,title,company,link,job_desc,job_func_stand,location_standardized,job_type_stand,source
0,Junior Web Developer,INQUIRER.net,https://ph.linkedin.com/jobs/view/junior-web-d...,\r\n <strong>Key Responsibilities<br>...,"Engineering, Information Technology","Manila City, National Capital Region (NCR)",Full-time,Linkedin
1,Junior Python Developer,ShipERP,https://ph.linkedin.com/jobs/view/junior-pytho...,\r\n <strong>Position Overview<br><br...,"Engineering, Information Technology",Philippines,Full-time,Linkedin
2,Python Developer with Data Engineering experie...,ERNI,https://ph.linkedin.com/jobs/view/python-devel...,\r\n Founded in 1994 and headquartere...,"Engineering, Information Technology","Manila City, National Capital Region (NCR)",Full-time,Linkedin
3,Junior Back End Developer,Media Meter Inc.,https://ph.linkedin.com/jobs/view/junior-back-...,\r\n <strong>Is this role right for y...,"Engineering, Information Technology","Quezon City, National Capital Region (NCR)",Full-time,Linkedin
4,Python Developer,Theoria Medical,https://ph.linkedin.com/jobs/view/python-devel...,\r\n <p><strong>Position Type</strong...,"Engineering, Information Technology","Manila City, National Capital Region (NCR)",Full-time,Linkedin
...,...,...,...,...,...,...,...,...
117,Software Engineer,N-PAX Group,https://ph.jobstreet.com/job/83142234?type=sta...,<p>Job description:<br>* Collaborate with a te...,Information Technology,"Manila City, National Capital Region (NCR)",Full-time,Jobstreet
118,Software Engineer,KYOCERA Document Solutions Development Philipp...,https://ph.jobstreet.com/job/82609090?type=sta...,<p><strong>KYOCERA DOCUMENT SOLUTIONS DEVELOPM...,Information Technology,"Cebu City, Central Visayas (Region VII)",Full-time,Jobstreet
119,Junior Software Engineer,"CHAMP Cargosystems Philippines, Inc.",https://ph.jobstreet.com/job/82868040?type=sta...,<p><strong>Who we are</strong></p><p>CHAMP Car...,Information Technology,"Manila City, National Capital Region (NCR)",Full-time,Jobstreet
120,Software Engineer,SPi Global (now Straive),https://ph.jobstreet.com/job/82583396?type=sta...,<p>Join our dynamic team to develop and mainta...,Information Technology,"Calamba City, CALABARZON (Region IV-A)",Full-time,Jobstreet


## Dropping Duplicates

In [99]:
duplicates = job_listings[job_listings.duplicated(subset=['title', 'company'], keep=False)]
duplicates = duplicates.groupby(['title', 'company','source']).size().reset_index(name='count')
duplicates = duplicates.sort_values(by='count', ascending=False)
duplicates

,title,company,source,count
9,Python Developer,Theoria Medical,Linkedin,15
10,Senior Android Developer,LeapFroggr Inc.,Kalibrr,4
0,Full Stack Developer (React/Node/Python) - Nig...,MicroSourcing,Foundit,2
3,Full-Stack Developer (Python/Django &amp; React),One Outsource,Foundit,2
11,Software Developer (Python),"Risewave Consulting, Inc.",Foundit,2
1,Full Stack Python Developer,Tahche Careers,Foundit,1
2,Full Stack Python Developer,Tahche Careers,Linkedin,1
4,Junior Python Developer,ShipERP,Foundit,1
5,Junior Python Developer,ShipERP,Linkedin,1
6,Python Developer,Genzeon,Foundit,1


In [100]:
# retain first listing
job_listings = job_listings.drop_duplicates(subset=['title', 'company'], keep='first')
job_listings

,title,company,link,job_desc,job_func_stand,location_standardized,job_type_stand,source
0,Junior Web Developer,INQUIRER.net,https://ph.linkedin.com/jobs/view/junior-web-d...,\r\n <strong>Key Responsibilities<br>...,"Engineering, Information Technology","Manila City, National Capital Region (NCR)",Full-time,Linkedin
1,Junior Python Developer,ShipERP,https://ph.linkedin.com/jobs/view/junior-pytho...,\r\n <strong>Position Overview<br><br...,"Engineering, Information Technology",Philippines,Full-time,Linkedin
2,Python Developer with Data Engineering experie...,ERNI,https://ph.linkedin.com/jobs/view/python-devel...,\r\n Founded in 1994 and headquartere...,"Engineering, Information Technology","Manila City, National Capital Region (NCR)",Full-time,Linkedin
3,Junior Back End Developer,Media Meter Inc.,https://ph.linkedin.com/jobs/view/junior-back-...,\r\n <strong>Is this role right for y...,"Engineering, Information Technology","Quezon City, National Capital Region (NCR)",Full-time,Linkedin
4,Python Developer,Theoria Medical,https://ph.linkedin.com/jobs/view/python-devel...,\r\n <p><strong>Position Type</strong...,"Engineering, Information Technology","Manila City, National Capital Region (NCR)",Full-time,Linkedin
...,...,...,...,...,...,...,...,...
117,Software Engineer,N-PAX Group,https://ph.jobstreet.com/job/83142234?type=sta...,<p>Job description:<br>* Collaborate with a te...,Information Technology,"Manila City, National Capital Region (NCR)",Full-time,Jobstreet
118,Software Engineer,KYOCERA Document Solutions Development Philipp...,https://ph.jobstreet.com/job/82609090?type=sta...,<p><strong>KYOCERA DOCUMENT SOLUTIONS DEVELOPM...,Information Technology,"Cebu City, Central Visayas (Region VII)",Full-time,Jobstreet
119,Junior Software Engineer,"CHAMP Cargosystems Philippines, Inc.",https://ph.jobstreet.com/job/82868040?type=sta...,<p><strong>Who we are</strong></p><p>CHAMP Car...,Information Technology,"Manila City, National Capital Region (NCR)",Full-time,Jobstreet
120,Software Engineer,SPi Global (now Straive),https://ph.jobstreet.com/job/82583396?type=sta...,<p>Join our dynamic team to develop and mainta...,Information Technology,"Calamba City, CALABARZON (Region IV-A)",Full-time,Jobstreet


In [101]:
job_listings = job_listings.reset_index().drop('index', axis=1)
job_listings.columns = ['title', 'company', 'link','job_desc','job_func','location','job_type','source']

## Cleaning Job Description

In [113]:
def remove_html_tags(text):
    clean = re.sub(r'<.*?>', '', text)
    clean = clean.replace('\n', '').replace('\r', '').strip()
    return clean

In [125]:
job_listings['job_desc'] = job_listings.apply(lambda row: remove_html_tags(row['job_desc']), axis=1)
job_listings.head()

,title,company,link,job_desc,job_func,location,job_type,source
0,Junior Web Developer,INQUIRER.net,https://ph.linkedin.com/jobs/view/junior-web-d...,Key Responsibilities Front-End Development and...,"Engineering, Information Technology","Manila City, National Capital Region (NCR)",Full-time,Linkedin
1,Junior Python Developer,ShipERP,https://ph.linkedin.com/jobs/view/junior-pytho...,Position OverviewAs an Entry-Level Python Deve...,"Engineering, Information Technology",Philippines,Full-time,Linkedin
2,Python Developer with Data Engineering experie...,ERNI,https://ph.linkedin.com/jobs/view/python-devel...,Founded in 1994 and headquartered in Switzerla...,"Engineering, Information Technology","Manila City, National Capital Region (NCR)",Full-time,Linkedin
3,Junior Back End Developer,Media Meter Inc.,https://ph.linkedin.com/jobs/view/junior-back-...,Is this role right for you?What are your respo...,"Engineering, Information Technology","Quezon City, National Capital Region (NCR)",Full-time,Linkedin
4,Python Developer,Theoria Medical,https://ph.linkedin.com/jobs/view/python-devel...,"Position Type: Full-Time, Non-exemptCompensati...","Engineering, Information Technology","Manila City, National Capital Region (NCR)",Full-time,Linkedin


# Checkpoint

In [299]:
job_listings.to_csv('job_listings.csv')

In [300]:
job_listings = pd.read_csv('job_listings.csv').drop('Unnamed: 0', axis=1)
job_listings.head()

,title,company,link,job_desc,job_func,location,job_type,source
0,Junior Web Developer,INQUIRER.net,https://ph.linkedin.com/jobs/view/junior-web-d...,Key Responsibilities Front-End Development and...,"Engineering, Information Technology","Manila City, National Capital Region (NCR)",Full-time,Linkedin
1,Junior Python Developer,ShipERP,https://ph.linkedin.com/jobs/view/junior-pytho...,Position OverviewAs an Entry-Level Python Deve...,"Engineering, Information Technology",Philippines,Full-time,Linkedin
2,Python Developer with Data Engineering experie...,ERNI,https://ph.linkedin.com/jobs/view/python-devel...,Founded in 1994 and headquartered in Switzerla...,"Engineering, Information Technology","Manila City, National Capital Region (NCR)",Full-time,Linkedin
3,Junior Back End Developer,Media Meter Inc.,https://ph.linkedin.com/jobs/view/junior-back-...,Is this role right for you?What are your respo...,"Engineering, Information Technology","Quezon City, National Capital Region (NCR)",Full-time,Linkedin
4,Python Developer,Theoria Medical,https://ph.linkedin.com/jobs/view/python-devel...,"Position Type: Full-Time, Non-exemptCompensati...","Engineering, Information Technology","Manila City, National Capital Region (NCR)",Full-time,Linkedin


# Utility Matrix

In [301]:
matrix = job_listings[['title', 'job_desc', 'job_func', 'location', 'job_type']]

In [304]:
def location_to_coord(df_target, df_locations):
    df_locations['location'] = df_locations['City/Province'] + ', ' + df_locations['Region']
    df_target = df_target.merge(df_locations[['location', 'Latitude', 'Longitude']], on='location', how='left')
    df_target = df_target.drop('location', axis=1)
    df_target['Latitude'] = df_target['Latitude'].fillna(12.8797)
    df_target['Longitude'] = df_target['Longitude'].fillna(121.7740)
    return df_target

def job_type_encoding(df_target):
    one_hat = pd.get_dummies(df_target['job_type'])
    for col in ['Full-time', 'Contract/Temporary', 'Part-time', 'Internship']:
        if col not in one_hat.columns:
            one_hat[col] = False
    one_hat = one_hat[['Full-time', 'Contract/Temporary', 'Part-time', 'Internship']]
    df_target = pd.concat([df_target, one_hat.astype(int)], axis=1)
    df_target = df_target.drop('job_type', axis=1)
    return df_target

def sentence_2_vec(df_target):
    df_target = df_target.fillna({'job_desc': '', 'job_func': ''})
    model = SentenceTransformer("all-MiniLM-L6-v2")
    embeddings = pd.DataFrame(model.encode(df_target['job_desc']))
    embeddings2 = pd.DataFrame(model.encode(df_target['job_func']))
    embeddings2.columns = [x+len(embeddings.columns) for x in embeddings2.columns]
    df_target = df_target.drop(['job_desc', 'job_func'], axis=1)
    df_target = pd.concat([df_target, embeddings, embeddings2], axis=1)
    return df_target

In [305]:
matrix = job_type_encoding(matrix)
matrix = location_to_coord(matrix, df_locations)
matrix = sentence_2_vec(matrix)

In [306]:
matrix

,title,Full-time,Contract/Temporary,Part-time,Internship,Latitude,Longitude,0,1,2,...,758,759,760,761,762,763,764,765,766,767
0,Junior Web Developer,1,0,0,0,14.599500,120.98420,-0.020892,-0.010042,-0.023996,...,0.035966,0.039613,0.034523,-0.052508,-0.043252,0.039149,0.056286,-0.075177,0.094946,0.039914
1,Junior Python Developer,1,0,0,0,12.879700,121.77400,-0.043462,-0.005527,0.011338,...,0.035966,0.039613,0.034523,-0.052508,-0.043252,0.039149,0.056286,-0.075177,0.094946,0.039914
2,Python Developer with Data Engineering experie...,1,0,0,0,14.599500,120.98420,-0.025429,0.073876,0.015897,...,0.035966,0.039613,0.034523,-0.052508,-0.043252,0.039149,0.056286,-0.075177,0.094946,0.039914
3,Junior Back End Developer,1,0,0,0,14.676200,121.04390,0.063229,0.005469,0.009441,...,0.035966,0.039613,0.034523,-0.052508,-0.043252,0.039149,0.056286,-0.075177,0.094946,0.039914
4,Python Developer,1,0,0,0,14.599500,120.98420,0.002482,0.007215,0.005364,...,0.035966,0.039613,0.034523,-0.052508,-0.043252,0.039149,0.056286,-0.075177,0.094946,0.039914
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,Software Engineer,1,0,0,0,14.599500,120.98420,-0.013973,-0.006533,-0.035216,...,0.031285,-0.023702,0.100298,0.000189,0.006308,0.064320,0.056485,-0.074071,0.074240,0.010131
94,Software Engineer,1,0,0,0,10.315700,123.88540,-0.104513,-0.013462,0.025250,...,0.031285,-0.023702,0.100298,0.000189,0.006308,0.064320,0.056485,-0.074071,0.074240,0.010131
95,Junior Software Engineer,1,0,0,0,14.599500,120.98420,-0.058654,-0.017591,-0.038974,...,0.031285,-0.023702,0.100298,0.000189,0.006308,0.064320,0.056485,-0.074071,0.074240,0.010131
96,Software Engineer,1,0,0,0,14.192048,121.13118,-0.026431,-0.016797,-0.004591,...,0.031285,-0.023702,0.100298,0.000189,0.006308,0.064320,0.056485,-0.074071,0.074240,0.010131


# User Profile

In [455]:
location = "Biñan City, CALABARZON (Region IV-A)"
job_desc = "60,000 salary, python developer, data science"
job_func = "Software Engineer"
job_type = "Full-time"
job_query = "python developer"

user = pd.DataFrame({'job_desc':[job_desc], 'job_func':[job_func], 'location':[location], 'job_type':[job_type]})

In [308]:
user_profile = job_type_encoding(user)
user_profile = location_to_coord(user_profile, df_locations)
user_profile = sentence_2_vec(user_profile)

# Recommending the Jobs

In [540]:
def compute_similarity_score(user, matrix):
    job_func_desc_score = cosine_similarity(user_profile.iloc[:,6:], matrix.iloc[:,7:])
    location_score = 1/(1+euclidean_distances(user_profile[['Latitude','Longitude']],matrix[['Latitude','Longitude']]))
    job_type_score = np.array([matrix.apply(lambda row: np.dot(user_profile.iloc[:,:4].values[0], row[1:5].values), axis=1)])
    title_score = np.array(matrix.apply(lambda row: normalized_similarity(job_query, row['title']), axis=1))
    
    total = job_func_desc_score + location_score + job_type_score + title_score
    return pd.DataFrame({"total":total[0]})

In [552]:
total = compute_similarity_score(user_profile, matrix)
job_listings.iloc[total.sort_values('total', ascending=False).index]

,title,company,link,job_desc,job_func,location,job_type,source
15,Python Developer,JK Network Services,https://ph.linkedin.com/jobs/view/python-devel...,Company Profile: One of the largest BPO consul...,"Engineering, Information Technology","Pasay City, National Capital Region (NCR)",Full-time,Linkedin
14,Python Developer,"World Networking Services Philippines, Inc",https://ph.linkedin.com/jobs/view/python-devel...,"A bachelor's degree in mathematics, engineerin...","Engineering, Information Technology","Pasig City, National Capital Region (NCR)",Full-time,Linkedin
4,Python Developer,Theoria Medical,https://ph.linkedin.com/jobs/view/python-devel...,"Position Type: Full-Time, Non-exemptCompensati...","Engineering, Information Technology","Manila City, National Capital Region (NCR)",Full-time,Linkedin
60,Python Developer,VISEO ASIA,https://www.foundit.com.ph/job/python-develope...,We are seeking a talented Python Developer wit...,NaN,"Manila City, National Capital Region (NCR)",Full-time,Foundit
45,Python Developer,Bryd,https://www.foundit.com.ph/job/python-develope...,About the JobWe&amp;aposre looking for a Pytho...,NaN,"Manila City, National Capital Region (NCR)",Full-time,Foundit
...,...,...,...,...,...,...,...,...
51,Jr to Mid .NET or Python Developer with Flutte...,Outsourced,https://www.foundit.com.ph/job/jr-to-mid-net-o...,Job SummaryJr to Mid .NET or Python Developer ...,NaN,Philippines,Full-time,Foundit
92,Mid-level Back-end Developer,N-Compass TV Rep. Office,https://ph.jobstreet.com/job/83147320?type=sta...,We’re looking for a Mid-level Backend Develope...,Information Technology,"Cebu City, Central Visayas (Region VII)",Full-time,Jobstreet
94,Software Engineer,KYOCERA Document Solutions Development Philipp...,https://ph.jobstreet.com/job/82609090?type=sta...,KYOCERA DOCUMENT SOLUTIONS DEVELOPMENT PHILIPP...,Information Technology,"Cebu City, Central Visayas (Region VII)",Full-time,Jobstreet
10,"Python Developer (AI, Python &amp; OOP)","Risewave Consulting, Inc.",https://ph.linkedin.com/jobs/view/python-devel...,The Role• Work with the global Calculation Dev...,Information Technology,"Taguig City, National Capital Region (NCR)",Contract/Temporary,Linkedin
